In [21]:
from PIL import Image
import torchvision.transforms as transforms
import torch

### Importing custom modules

#### Importing helper

In [22]:
import sys
sys.path.append('./helper')

#### Importing model

In [23]:
sys.path.append('./model/template')
import faster_r_cnn

### Training model

#### Defining Deep Fashion 2 Dataset

In [24]:
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor()
])

In [25]:
class DeepFashion2Dataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, json_paths):
        self.image_paths = image_paths
        self.json_paths = json_paths

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert('RGB')
        image_tensor = transform(image)

        target_tensor = {
            "boxes": torch.tensor(bbox, dtype=torch.float32),  
            "labels" : torch.tensor(labels, dtype=torch.int64)
        }

        return image_tensor, target_tensor

    def __len__(self):
        return len(self.image_paths)

In [26]:
start_index = 1
end_index = 191161

data_ids = [str(i).zfill(6) for i in range(start_index, end_index + 1)]
image_paths = ["../data/train/image/" + data_id + ".jpg" for data_id in data_ids]
json_paths = ["../data/train/annos/" + data_id + ".json" for data_id in data_ids]

In [27]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = DeepFashion2Dataset(image_paths, json_paths)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True,collate_fn=collate_fn)

#### Setting Model

In [28]:
model = faster_r_cnn.FasterRCNNResNet101(num_classes=13)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

FasterRCNNResNet101(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=1e-05)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=1e-05)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=1e-05)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [29]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#### Looping train

In [30]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    
    for images, targets in train_dataloader:        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass and optimization
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

    # Step the learning rate scheduler
    lr_scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader)}")

print("Training complete!")

IndexError: list index out of range

#### Saving trained model

In [ ]:
torch.save(model.state_dict(), "./model/saved/faster_rcnn_resnet101.pth")